# Exercise 1: Create an imputer using the 'most frequent' strategy

Previously, we used the following code:

```python
imputer = SimpleImputer(strategy = "mean")
```

I effectively declared that I wanted to use the standard strategy for imputing data (i.e., replacing all `NaNs` with the mean of each column). In this exercise, I want you to redo what we have done above, but this time create an imputer that replaces the values with the 'most frequent' value. To do this, you will need to:

1. Split the data again. Above I already imputed the data for `train_set` and `test_set` so you need to create these sets again from the original `data` dataframe.
2. Read about how to change the strategy to meet your goals in the [sklearn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html).
3. Create and `.fit()` a new `SimpleImputer()`
4. Apply this new imputer to your new `train_set` and `test_set`

You can do all of this in one cell of code if you want (and its actually recommended to try and do it this way!).

In [1]:
import numpy as np
import pandas as pd

In [3]:
# Good practice to import all necessary modules at the top of cell so that your users
# can see what modules are required for your code to run
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# Read data
data = pd.read_csv('Data/core_data.csv')

# Drop duplicates
data.drop_duplicates(inplace = True)

# Split the data - a 70:30 train:test split is generally recommended, but it can be adjusted depending
# on the size of the dataset
train_set, test_set = train_test_split(data, train_size = 0.7, random_state = 42)

# Train an imputer using the appropriate method
imputer = SimpleImputer(strategy = "most_frequent")

# Here we train the imputer ONLY on the numeric data
num_cols = train_set.select_dtypes(include = np.number).columns
imputer.fit(train_set[num_cols]) # the .fit() method trains the imputer on the data we give to it

# Transform the train and test sets using our trained imputer
# Remember that because we trained our imputer on ONLY the numeric columns, we must give the imputer
# datasets that contain ONLY those columns
train_set[num_cols] = imputer.transform(train_set[num_cols])
test_set[num_cols] = imputer.transform(test_set[num_cols])

# Check to see if there are any missing values
train_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3528 entries, 1152 to 949
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   label                   3528 non-null   object 
 1   Depth CSF-A (m)         3528 non-null   float64
 2   Porosity (vol%)         3528 non-null   float64
 3   NGR total counts (cps)  3528 non-null   float64
 4   Reflectance L*          3528 non-null   float64
 5   Reflectance a*          3528 non-null   float64
 6   Reflectance b*          3528 non-null   float64
dtypes: float64(6), object(1)
memory usage: 220.5+ KB


# Exercise 2

So far, we've conducted a lot of data preparation on the training set. However, we also need to need to apply this to the test set (as well as to any new set of unknown data). So how are we going to do that? It would be impractical and difficult to do this manually and would be very prone to errors.

Instead, in this exercise you are going to write three functions to do this programmatically:

* `split_dataframe(df)`: this function will take the entire dataframe, remove duplicates, and return two dataframes (a `train_set`, and a `test_set`) 
* `fit_train_parameters(train_set)`: this function will take the `train_set` as an input, and apply all of the imputation and scaling strategies that we've used above. Here, I want you to use a `SimpleImputer` using the mean as your strategy. Once all the missing values have been replaced by the imputer, I want the function to scale the data using a normalization strategy. Read about the `MinMaxScaler()` class in `Sklearn` and use it on your data. The function should return a tuple containing the **trained** imputer and scaler objects.
* `transform_data(parameters, df)` will impute and scale any data (`df`) passed to it. The `parameters` argument should take the tuple of imputer and scaler objects and apply them to the passed dataframe.

If you've done everything correctly, the entire workflow of this lecture will be shortened to the following:

```python
data = pd.read_csv('Datasets/core_data.csv')
train_set, test_set = split_dataframe(data)
parameters = fit_train_parameters(train_set)
train_set = transform_data(parameters, train_set)
test_set = transform_data(parameters, test_set)
```

Have fun!

First, let's reload a clean version of the dataset:

In [6]:
data = pd.read_csv('Data/core_data.csv')

data.head()

,label,Depth CSF-A (m),Porosity (vol%),NGR total counts (cps),Reflectance L*,Reflectance a*,Reflectance b*
0,317-U1351A-001H-1-0.265,0.26,57.0,23.219,43.0,-2.5,10.1
1,342-U1409A-001H-1-0.26,0.26,79.9,20.350,53.4,1.5,1.9
2,342-U1409A-001H-1-0.26,0.26,79.9,20.350,53.4,1.5,1.9
3,342-U1408A-001H-1-0.37,0.37,64.1,23.010,47.9,3.2,6.1
4,342-U1407A-001H-1-0.38,0.38,54.9,24.460,51.3,4.7,8.4


In [7]:
# Your Code here
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

def split_dataframe(df):
    '''
    Drops any duplicates in the dataframe and then splits the dataframe into a train and test set using a 70:30 train:test split
    '''
    df.drop_duplicates(inplace = True)
    train_set, test_set = train_test_split(df, train_size = 0.7, random_state = 42)
    return train_set, test_set
    
def fit_train_parameters(train_set):
    '''
    Trains an imputer and scaler object that will be used to transform the data    
    '''
    # We use default arguments, but you can change these to fit your needs
    imputer = SimpleImputer()
    scaler = MinMaxScaler()
    
    # Create a list of the numeric columns
    num_cols = train_set.select_dtypes(include = np.number).columns
    
    # Train the imputer on the numeric columns of the train_set
    imputer.fit(train_set[num_cols])
    
    # Transform the train_set using our newly trained imputer
    # We do this because when we train our scaler, we want to ensure that its trained on all the values
    train_set[num_cols] = imputer.transform(train_set[num_cols])
    
    # Train our scaler on the numeric columns of the train_set (which now does not have any missing values)
    scaler.fit(train_set[num_cols])
    
    # Note that we do not need to transform the train set using our scaler, since all we want to do in this
    # function is return the trained imputer and scaler objects
    return imputer, scaler
    
def transform_data(parameters, df):
    '''
    Transforms a dataframe according to the trained imputer and scaler objects
    '''
    # Extract our imputer and scaler objects from the parameters tuple
    # imputer, scaler = parameters <- this also works
    imputer = parameters[0]
    scaler = parameters[1]
    
    # Once again, create a list of the numeric columns
    num_cols = train_set.select_dtypes(include = np.number).columns
    
    # Remember that because we trained our imputer/scaler on ONLY the numeric columns, we must give the imputer/scaler
    # datasets that contain ONLY those columns
    df[num_cols] = imputer.transform(df[num_cols])
    df[num_cols] = scaler.transform(df[num_cols])
    
    # Return the transformed dataframe
    return df

def remove_negative_values(df):
    '''
    Remove any negative values in the two culprit columns
    '''
    df = df[df['Porosity (vol%)'] > 0]
    df = df[df['NGR total counts (cps)'] > 0]
    
    # Return dataframe with no negative values
    return df

In [9]:
data = pd.read_csv('Data/core_data.csv')

# Split our data using our function
train_set, test_set = split_dataframe(data)

# Remove negative values from both train and test sets
train_set = remove_negative_values(train_set)
test_set = remove_negative_values(test_set)

# Create our trained imputer/scaler objects
parameters = fit_train_parameters(train_set)

# Transform both train and test sets using our trained imputer/scaler objects
train_set = transform_data(parameters, train_set)
test_set = transform_data(parameters, test_set)

In [10]:
# Check to make sure it worked
display(train_set.describe())
display(train_set.info())
display(test_set.describe())
display(test_set.info())

,Depth CSF-A (m),Porosity (vol%),NGR total counts (cps),Reflectance L*,Reflectance a*,Reflectance b*
count,3521.000000,3521.000000,3521.000000,3521.000000,3521.000000,3521.000000
mean,0.156829,0.591335,0.313595,0.472538,0.333359,0.370323
std,0.192779,0.192094,0.167324,0.133532,0.081457,0.086184
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.037179,0.482436,0.201632,0.382620,0.282392,0.304207
50%,0.088735,0.597190,0.280822,0.460830,0.310299,0.359223
75%,0.196034,0.724824,0.406729,0.547730,0.365449,0.432039
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3521 entries, 1152 to 949
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   label                   3521 non-null   object 
 1   Depth CSF-A (m)         3521 non-null   float64
 2   Porosity (vol%)         3521 non-null   float64
 3   NGR total counts (cps)  3521 non-null   float64
 4   Reflectance L*          3521 non-null   float64
 5   Reflectance a*          3521 non-null   float64
 6   Reflectance b*          3521 non-null   float64
dtypes: float64(6), object(1)
memory usage: 220.1+ KB


None

,Depth CSF-A (m),Porosity (vol%),NGR total counts (cps),Reflectance L*,Reflectance a*,Reflectance b*
count,1512.000000,1512.000000,1512.000000,1512.000000,1512.000000,1512.000000
mean,0.151069,0.597059,0.309879,0.471096,0.331608,0.368261
std,0.190665,0.197083,0.167222,0.131716,0.081701,0.085437
min,0.000104,0.000000,0.006869,0.023346,0.156146,-0.027508
25%,0.033301,0.482143,0.198961,0.380026,0.282392,0.304207
50%,0.084324,0.608899,0.276204,0.461284,0.308970,0.352751
75%,0.189012,0.734485,0.394325,0.544877,0.362126,0.429733
max,1.001447,1.010539,1.013182,1.024643,0.926910,0.855987


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1512 entries, 5433 to 604
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   label                   1512 non-null   object 
 1   Depth CSF-A (m)         1512 non-null   float64
 2   Porosity (vol%)         1512 non-null   float64
 3   NGR total counts (cps)  1512 non-null   float64
 4   Reflectance L*          1512 non-null   float64
 5   Reflectance a*          1512 non-null   float64
 6   Reflectance b*          1512 non-null   float64
dtypes: float64(6), object(1)
memory usage: 94.5+ KB


None


# Exercise 3

In this exercise, we will repeat what we did in exercise 2 but this time using the `Pipeline` class in `sklearn`. Create a data preparation pipeline that takes your numerical data in the dataframe (ignore the non-numerical data - try to use `data.dtypes`), replace all the `NaNs` with the `mean` values, and normalises your data (i.e. using the `MinMaxScaler()` class). 

Note: the `Pipeline` class can only use `sklearn` Transformer and Model objects - you won't be able to put your `split_dataframe()` function into the Pipeline (it can be done, but is outside of the scope of this course - you can look up how to create custom sklearn Transformers if you're interested!). Instead, you will need to start from the `data` dataframe and redo the `train_test_split` (feel free to reuse your previous functions from Exercise 2) before putting the data in the pipeline. Then apply your pipeline to your new train and test sets.

In [11]:
from sklearn.pipeline import Pipeline

# Create a pipeline object with an imputer and a scaler
my_pipeline = Pipeline([
    ('imputer', SimpleImputer()),
    ('scaler', MinMaxScaler())
])

# When you give data to the pipeline, you are feeding that data through the imputer followed by the scaler
my_pipeline

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', MinMaxScaler())])

In [12]:
# First thing to do is write out our full workflow to make sure we know all the steps
# - read data
# - drop duplicates
# - split data into train/test
# - remove negative values
# - train imputer/scaler
# - transform train/test using imputer/scaler

# Read data
data = pd.read_csv('Data/core_data.csv')

# Split our data using our function
train_set, test_set = split_dataframe(data)

# Remove negative values from both train and test sets
train_set = remove_negative_values(train_set)
test_set = remove_negative_values(test_set)

# Now we want to train our pipeline object on the train_set
num_cols = train_set.select_dtypes(include = np.number).columns
my_pipeline.fit(train_set[num_cols]) # trained on numeric columns

# We can now transform our data using our pipeline object
# This step will impute missing values AND scale our data in one line of code
train_set[num_cols] = my_pipeline.transform(train_set[num_cols])
test_set[num_cols] = my_pipeline.transform(test_set[num_cols])

# Note that the following lines of code:
#      my_pipeline.fit(train_set[num_cols]) 
#      train_set[num_cols] = my_pipeline.transform(train_set[num_cols])
# Can be combined into a single line of code:
#      train_set[num_cols] = my_pipeline.fit_transform(train_set[num_cols])

In [13]:
# Check to make sure it worked
display(train_set.describe())
display(train_set.info())
display(test_set.describe())
display(test_set.info())

,Depth CSF-A (m),Porosity (vol%),NGR total counts (cps),Reflectance L*,Reflectance a*,Reflectance b*
count,3521.000000,3521.000000,3521.000000,3521.000000,3521.000000,3521.000000
mean,0.156829,0.591335,0.313595,0.472538,0.333359,0.370323
std,0.192779,0.192094,0.167324,0.133532,0.081457,0.086184
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.037179,0.482436,0.201632,0.382620,0.282392,0.304207
50%,0.088735,0.597190,0.280822,0.460830,0.310299,0.359223
75%,0.196034,0.724824,0.406729,0.547730,0.365449,0.432039
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3521 entries, 1152 to 949
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   label                   3521 non-null   object 
 1   Depth CSF-A (m)         3521 non-null   float64
 2   Porosity (vol%)         3521 non-null   float64
 3   NGR total counts (cps)  3521 non-null   float64
 4   Reflectance L*          3521 non-null   float64
 5   Reflectance a*          3521 non-null   float64
 6   Reflectance b*          3521 non-null   float64
dtypes: float64(6), object(1)
memory usage: 220.1+ KB


None

,Depth CSF-A (m),Porosity (vol%),NGR total counts (cps),Reflectance L*,Reflectance a*,Reflectance b*
count,1512.000000,1512.000000,1512.000000,1512.000000,1512.000000,1512.000000
mean,0.151069,0.597059,0.309879,0.471096,0.331608,0.368261
std,0.190665,0.197083,0.167222,0.131716,0.081701,0.085437
min,0.000104,0.000000,0.006869,0.023346,0.156146,-0.027508
25%,0.033301,0.482143,0.198961,0.380026,0.282392,0.304207
50%,0.084324,0.608899,0.276204,0.461284,0.308970,0.352751
75%,0.189012,0.734485,0.394325,0.544877,0.362126,0.429733
max,1.001447,1.010539,1.013182,1.024643,0.926910,0.855987


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1512 entries, 5433 to 604
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   label                   1512 non-null   object 
 1   Depth CSF-A (m)         1512 non-null   float64
 2   Porosity (vol%)         1512 non-null   float64
 3   NGR total counts (cps)  1512 non-null   float64
 4   Reflectance L*          1512 non-null   float64
 5   Reflectance a*          1512 non-null   float64
 6   Reflectance b*          1512 non-null   float64
dtypes: float64(6), object(1)
memory usage: 94.5+ KB


None